Following is the code to do a basic check for the examples that we are passing
It will be checking if it is able to open and read the file and will also give the length of role messages we have included.
In the below code I am giving 11 examples, because if it is less than 10, the finetuning job returned an error saying that it needs to be atleast 10 examples.

In [3]:
import json
def basic_checks(data_file):
    try:
        with open(data_file, 'r', encoding='utf-8') as f:
            dataset = [json.loads(line) for line in f]
        print(f"Basic checks for file {data_file}:")
        print("Count of examples in training dataset:", len(dataset))
        print("First example:")
        for message in dataset[0]["messages"]:
            print(message)
        return True
    except Exception as e:
        print(f"An error occurred in file {data_file}: {e}")
        return False

basic_checks(f'C:\\Users\\Riaz\\Desktop\\MSDS\\Generative AI\\Week 7\\data_file')

Basic checks for file C:\Users\Riaz\Desktop\MSDS\Generative AI\Week 7\data_file:
Count of examples in training dataset: 11
First example:
{'role': 'system', 'content': "You're a chatbot that only responds with emojis!"}
{'role': 'user', 'content': 'I heard the koala habitat was destroyed by the fire.'}
{'role': 'assistant', 'content': '(sadkoala)'}


True

In [5]:
import os
import requests
import json
import openai
#import openai
from openai import OpenAI
openai.api_key =os.getenv("openai_api_key")

The created file is being uploaded using the files API, this is required for the fine tuning job to pick the training file.

In [7]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("C:\\Users\\Riaz\\Desktop\\MSDS\\Generative AI\\Week 7\\data_file", "rb"),
  purpose="fine-tune"
)

print ("The information of uploaded files\n",client.files.list())


The information of uploaded files
 SyncCursorPage[FileObject](data=[FileObject(id='file-FNdXMDgfG35syu5VTKvcft', bytes=2242, created_at=1737861110, filename='data_file', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-RffFCTQefajKoKMd7AaesC', bytes=2644, created_at=1737749530, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None), FileObject(id='file-GEJRrXbqseFqXxdUn6cF4Z', bytes=2242, created_at=1737749017, filename='data_file', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-J2jAiXJTExmrqfMVQ8fYeg', bytes=666, created_at=1737747122, filename='data_file', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-MFathCpeBf1sismJMxooHv', bytes=666, created_at=1737747089, filename='data_file', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file

Fine tuning job needs to be created with the parameter of file id and this is retrieved from the above files.list api.
The below command creates a finetuning job and the model gpt-4o-mini-2024-07-18 has been specified.  
The list of fine tunable models can be retrieved from https://platform.openai.com/docs/guides/fine-tuning

In [9]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(training_file="file-FNdXMDgfG35syu5VTKvcft",model="gpt-4o-mini-2024-07-18")

FineTuningJob(id='ftjob-LW7hiUMdjJBeaNKD8Ce9fBnx', created_at=1737861136, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-CICsZaJ3gjcM8Nb7O39cmoMG', result_files=[], seed=1271288324, status='validating_files', trained_tokens=None, training_file='file-FNdXMDgfG35syu5VTKvcft', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

Following is the command to get the list of running jobs and to retrieve the job details.
This is required, as it will give the fine tuned model.
While we use the chat completions API to query the LLM, this fine tuned model needs to be specified.
As we have not given the hyperparameters, it choses itself the optimimum values and displays the hyperparameters used like number of epochs, whether it is supervised or not and batch size.

In [25]:
client.fine_tuning.jobs.list(limit=10)
client.fine_tuning.jobs.retrieve('ftjob-LW7hiUMdjJBeaNKD8Ce9fBnx')

FineTuningJob(id='ftjob-LW7hiUMdjJBeaNKD8Ce9fBnx', created_at=1737861136, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AtnTcSYz', finished_at=1737861590, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=9), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-CICsZaJ3gjcM8Nb7O39cmoMG', result_files=['file-Ey9L51DRxJAKnRm7AWMut5'], seed=1271288324, status='succeeded', trained_tokens=3141, training_file='file-FNdXMDgfG35syu5VTKvcft', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=9)), type='supervised'), user_provided_suffix=None)

Alternatively, the running job information can also be retrieved using the Requests library of python

In [27]:
import requests

# Your OpenAI API key
api_key = os.getenv("openai_api_key")

# The fine-tune ID you want to check
fine_tune_id = "ftjob-LW7hiUMdjJBeaNKD8Ce9fBnx"

# Endpoint for checking the fine-tuning status
url = f"https://api.openai.com/v1/fine_tuning/jobs/{fine_tune_id}"

# Set up headers with your API key
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Make a GET request to the endpoint
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    # Parse and display the status information
    fine_tune_data = response.json()
    print("Fine-tune status:", fine_tune_data["status"])
    print("Details:", fine_tune_data)
else:
    print(f"Failed to fetch fine-tune status. Error: {response.status_code}")
    print("Response:", response.text)


Fine-tune status: succeeded
Details: {'object': 'fine_tuning.job', 'id': 'ftjob-LW7hiUMdjJBeaNKD8Ce9fBnx', 'model': 'gpt-4o-mini-2024-07-18', 'created_at': 1737861136, 'finished_at': 1737861590, 'fine_tuned_model': 'ft:gpt-4o-mini-2024-07-18:personal::AtnTcSYz', 'organization_id': 'org-CICsZaJ3gjcM8Nb7O39cmoMG', 'result_files': ['file-Ey9L51DRxJAKnRm7AWMut5'], 'status': 'succeeded', 'validation_file': None, 'training_file': 'file-FNdXMDgfG35syu5VTKvcft', 'hyperparameters': {'n_epochs': 9, 'batch_size': 1, 'learning_rate_multiplier': 1.8}, 'trained_tokens': 3141, 'error': {}, 'user_provided_suffix': None, 'seed': 1271288324, 'estimated_finish': None, 'integrations': [], 'method': {'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 9, 'batch_size': 1, 'learning_rate_multiplier': 1.8}}}}


The fine tuned model is being taken from the above output and used in the chat completions API to query.
It is seen that the output is showing as expected within brackets with no pictorial emoji displayed.

In [29]:
completion = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:personal::AtnTcSYz",
  messages=[
    {"role": "system", "content": "You're a chatbot that only responds with emojis!"},
    {"role": "user", "content": "What the hell is going on?"}
  ],
  max_tokens=50
)
print(completion.choices[0].message.content)

(surprised)


Comparing the above output, by passing the same information to an generic untrained model returns pictorial emojis

In [107]:
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You're a chatbot that only responds with emojis!"},
    {"role": "user", "content": "What the hell is going on?"}
  ],
  max_tokens=50
)
print(completion.choices[0].message.content)

😮🤔🔍
